<a href="https://colab.research.google.com/github/nguyenduyhieu201/machineLearningMidTerm/blob/main/MLMidTerm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [87]:
!ls /content/drive/MyDrive/Colab\ Notebooks/MLMidTerm.ipynb

'Bản sao của Colaboratory chào mừng bạn!'   MLMidTerm.ipynb


In [88]:
!git init
!git remote add origin https://github.com/nguyenduyhieu201/machineLearningMidTerm.git
!git config --global user.email "nguyenduyhieu202@gmail.com"
!git config --global user.name "Hieu"
!git add /content/drive/MyDrive/Colab\ Notebooks/MLMidTerm.ipynb
!git commit -m "Initial commit"
!git branch -M main
!git push -u origin main

Reinitialized existing Git repository in /content/.git/
fatal: remote origin already exists.
[master (root-commit) 96825d2] Initial commit
 1 file changed, 1 insertion(+)
 create mode 100644 drive/MyDrive/Colab Notebooks/MLMidTerm.ipynb
fatal: could not read Username for 'https://github.com': No such device or address


In [45]:
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
from PIL import Image
import torch
from torchvision import datasets, models, transforms
import torch.nn as nn
from torch.nn import functional as F
import torch.optim as optim
import torchvision
from torch.autograd import Variable
import time

import torch.nn as nn
 
input_path = "../content/DATA_CHAMBER_2021/"
use_gpu = torch.cuda.is_available()
if use_gpu:
    print("Using CUDA")


Using CUDA


In [46]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [47]:
!unzip -uq "/content/drive/My Drive/DATA_CHAMBER_2021.zip" -d "./"

In [48]:
class ImageFolderWithPaths(datasets.ImageFolder):
    # override the __getitem__ method. this is the method that dataloader calls
    def __getitem__(self, index):
        # this is what ImageFolder normally returns 
        original_tuple = super(ImageFolderWithPaths, self).__getitem__(index)
        # the image file path
        path = self.imgs[index][0]
        # make a new tuple that includes original and the path
        tuple_with_path = (original_tuple + (path,))
        return tuple_with_path

In [49]:
def data_transform(size):
  data_transforms = {
    'train':
    transforms.Compose([
        transforms.Resize(size),
        transforms.CenterCrop(size),
#         transforms.RandomAffine(0, shear=10, scale=(0.8,1.2)),
#         transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        normalize
    ]),
    'validation':
    transforms.Compose([
        transforms.Resize(size),
        transforms.CenterCrop(size),
        transforms.ToTensor(),
        normalize
    ]),
    }
  return data_transforms

In [50]:
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])


image_datasets = {
    'train': 
    ImageFolderWithPaths(input_path + 'train', data_transform(224)['train']),
    'validation': 
    ImageFolderWithPaths(input_path + 'test', data_transform(224)['validation'])
}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train','validation']}

dataloaders = {
    'train':
    torch.utils.data.DataLoader(image_datasets['train'],
                                batch_size=8,
                                shuffle=True,
                                num_workers=2),
    'validation':
    torch.utils.data.DataLoader(image_datasets['validation'],
                                batch_size=8,
                                shuffle=True,
                                num_workers=2)
}


In [51]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [52]:
!pip install efficientnet_pytorch 

In [53]:
modelResnet50 = models.resnet50(pretrained = True).cuda()
for param in modelResnet50.parameters():
    param.requires_grad = False           
modelResnet50.fc = nn.Sequential(
              nn.Linear(2048, 128),
              nn.ReLU(inplace=True),
              nn.Linear(128, 3)).to(device)
    
modelVgg16 = models.vgg16_bn(pretrained = True).cuda()
for param in modelVgg16.parameters():
  param.requires_grad = False
num_features = modelVgg16.classifier[6].in_features
modelVgg16.classifier[6] = nn.Linear(4096, 3).to(device)

modelResnet18 = models.resnet18(pretrained = True).cuda()
for param in modelResnet18.parameters():
    param.requires_grad = False           
modelResnet18.fc = nn.Sequential(
              nn.Linear(512, 128),
              nn.ReLU(inplace=True),
              nn.Linear(128, 3)).to(device)

In [54]:
criterion = nn.CrossEntropyLoss()
optimizerResnet50 = optim.SGD(modelResnet50.fc.parameters(),lr=0.001,momentum=0.9)
optimizerVgg16 = optim.SGD(modelVgg16.parameters(),lr=1,momentum=0.9)
optimizerResnet18 = optim.SGD(modelResnet18.fc.parameters(),lr=0.001,momentum=0.9)

In [55]:
def train_model(model, criterion, optimizer, num_epochs=3):
    train_batches = len(dataloaders['train'])
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch+1, num_epochs))
        print('-' * 10)

        for phase in ['train']:
            if phase == 'train':
                model.train()
            else:
                model.eval()

            running_loss = 0.0
            running_corrects = 0

            for i,data in enumerate(dataloaders[phase]):
                inputs, labels,_ = data
                print("\rTraining batch {}/{}".format(i+1, train_batches), end='', flush=True)
                inputs = inputs.to(device)
                labels = labels.to(device)

                outputs = model(inputs)
                loss = criterion(outputs, labels)

                if phase == 'train':
                    optimizer.zero_grad()
                    loss.backward()
                    optimizer.step()

                _, preds = torch.max(outputs, 1)
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(image_datasets[phase])
            epoch_acc = running_corrects.double() / len(image_datasets[phase])

            print('{} loss: {:.4f}, acc: {:.4f}'.format(phase,
                                                        epoch_loss,
                                                        epoch_acc))
    return model

In [60]:
model_trained_resnet50 = train_model(modelResnet50, criterion, optimizerResnet50, num_epochs=20)

Epoch 1/20
----------
Training batch 840/840train loss: 0.6287, acc: 0.7398
Epoch 2/20
----------
Training batch 840/840train loss: 0.5228, acc: 0.7902
Epoch 3/20
----------
Training batch 840/840train loss: 0.4565, acc: 0.8121
Epoch 4/20
----------
Training batch 840/840train loss: 0.4373, acc: 0.8218
Epoch 5/20
----------
Training batch 840/840train loss: 0.3668, acc: 0.8587
Epoch 6/20
----------
Training batch 840/840train loss: 0.3205, acc: 0.8732
Epoch 7/20
----------
Training batch 840/840train loss: 0.2929, acc: 0.8833
Epoch 8/20
----------
Training batch 840/840train loss: 0.2795, acc: 0.8919
Epoch 9/20
----------
Training batch 840/840train loss: 0.2473, acc: 0.9061
Epoch 10/20
----------
Training batch 840/840train loss: 0.2197, acc: 0.9180
Epoch 11/20
----------
Training batch 840/840train loss: 0.2002, acc: 0.9235
Epoch 12/20
----------
Training batch 840/840train loss: 0.1905, acc: 0.9308
Epoch 13/20
----------
Training batch 840/840train loss: 0.1714, acc: 0.9406
Epoch 14

In [61]:
model_trained_resnet18 = train_model(modelResnet18, criterion, optimizerResnet18, num_epochs=20)

Epoch 1/20
----------
Training batch 840/840train loss: 0.4025, acc: 0.8455
Epoch 2/20
----------
Training batch 840/840train loss: 0.3199, acc: 0.8800
Epoch 3/20
----------
Training batch 840/840train loss: 0.2736, acc: 0.8988
Epoch 4/20
----------
Training batch 840/840train loss: 0.2353, acc: 0.9134
Epoch 5/20
----------
Training batch 840/840train loss: 0.2205, acc: 0.9183
Epoch 6/20
----------
Training batch 840/840train loss: 0.1704, acc: 0.9428
Epoch 7/20
----------
Training batch 840/840train loss: 0.1511, acc: 0.9479
Epoch 8/20
----------
Training batch 840/840train loss: 0.1308, acc: 0.9595
Epoch 9/20
----------
Training batch 840/840train loss: 0.1122, acc: 0.9601
Epoch 10/20
----------
Training batch 840/840train loss: 0.0988, acc: 0.9708
Epoch 11/20
----------
Training batch 840/840train loss: 0.0955, acc: 0.9644
Epoch 12/20
----------
Training batch 840/840train loss: 0.0882, acc: 0.9698
Epoch 13/20
----------
Training batch 840/840train loss: 0.0897, acc: 0.9708
Epoch 14

In [62]:
model_trained_vgg16 = train_model(modelVgg16, criterion, optimizerVgg16, num_epochs=20)

Epoch 1/20
----------
Training batch 840/840train loss: 260.8878, acc: 0.5960
Epoch 2/20
----------
Training batch 840/840train loss: 280.4682, acc: 0.6531
Epoch 3/20
----------
Training batch 840/840train loss: 313.6347, acc: 0.6649
Epoch 4/20
----------
Training batch 840/840train loss: 282.5340, acc: 0.6926
Epoch 5/20
----------
Training batch 840/840train loss: 307.3144, acc: 0.6878
Epoch 6/20
----------
Training batch 840/840train loss: 289.6958, acc: 0.7005
Epoch 7/20
----------
Training batch 840/840train loss: 303.8634, acc: 0.7042
Epoch 8/20
----------
Training batch 840/840train loss: 307.9463, acc: 0.7024
Epoch 9/20
----------
Training batch 840/840train loss: 292.2456, acc: 0.7155
Epoch 10/20
----------
Training batch 840/840train loss: 277.3496, acc: 0.7292
Epoch 11/20
----------
Training batch 840/840train loss: 296.3367, acc: 0.7201
Epoch 12/20
----------
Training batch 840/840train loss: 305.3337, acc: 0.7216
Epoch 13/20
----------
Training batch 840/840train loss: 289.

In [65]:
def test_model(model, criterion, optimizer):
    labels_input=list()
    labels_output=list()
    vid_id = list()
    for phase in ['validation']:
        model.eval()

        running_loss = 0.0
        running_corrects = 0

        for inputs, labels, fname in dataloaders[phase]:
            inputs = inputs.to(device)
            labels = labels.to(device)
            labels_input= labels_input + labels.tolist()
            for f in fname:
                vid_id.append(f.split('/')[-1].split('.')[0].split('_')[0])
            outputs = model(inputs)
            
            loss = criterion(outputs, labels)
            _, preds = torch.max(outputs, 1)
            
            labels_output= labels_output + preds.tolist()
    return labels_input,labels_output,vid_id
resnet50_y_true,resnet50_y_pred,resnet50_vid_id = test_model(modelResnet50, criterion, optimizerResnet50)
resnet18_y_true,resnet18_y_pred,resnet18_vid_id = test_model(modelResnet18, criterion, optimizerResnet18)
vgg16_y_true,vgg16_y_pred,vgg16_vid_id = test_model(modelVgg16, criterion, optimizerVgg16)

In [71]:
from sklearn.metrics import confusion_matrix,accuracy_score,classification_report
print(classification_report(resnet50_y_true,resnet50_y_pred))
print(classification_report(resnet18_y_true,resnet18_y_pred))
print(classification_report(vgg16_y_true,vgg16_y_pred))
print('resnet 50 acc: ', accuracy_score(resnet50_y_true,resnet50_y_pred))
print('resnet 18 acc: ', accuracy_score(resnet18_y_true,resnet18_y_pred))
print('vgg16 acc: ', accuracy_score(vgg16_y_true,vgg16_y_pred))

              precision    recall  f1-score   support

           0       0.70      0.61      0.65       409
           1       0.52      0.81      0.63       367
           2       0.78      0.63      0.70       831

    accuracy                           0.67      1607
   macro avg       0.67      0.69      0.66      1607
weighted avg       0.70      0.67      0.67      1607

              precision    recall  f1-score   support

           0       0.53      0.75      0.62       409
           1       0.64      0.74      0.68       367
           2       0.90      0.67      0.77       831

    accuracy                           0.70      1607
   macro avg       0.69      0.72      0.69      1607
weighted avg       0.75      0.70      0.71      1607

              precision    recall  f1-score   support

           0       0.56      0.60      0.58       409
           1       0.52      0.75      0.62       367
           2       0.79      0.61      0.69       831

    accuracy        